# Scrape Wikipedia Page

#### Replicate Part One to Create Data Frame Starting Point

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 

In [2]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'lxml')

In [3]:
My_table = soup.find('table',{'class':'wikitable sortable'})

In [4]:
pcl = []
bl = []
nl= []

for row in My_table.findAll("tr"):
    cells = row.findAll("td")
    if len(cells) == 3:
        pc = cells[0].findAll(text=True)
        fpc = pc[0].replace("'", "")
        b = cells[1].findAll(text=True)
        fb = b[0].replace("'", "")
        n = cells[2].findAll(text=True)
        fn = n[0].replace("'", "")
        fn = fn.replace("\n", "")        
        if fb != "Not assigned":
            pcl.append(fpc)
            bl.append(fb)
            if fn != "Not assigned":
                nl.append(fn)
            if fn == "Not assigned":
                nl.append(fb)

In [5]:
l = len(pcl) - 1
NewPostalCodes =[]
RepeatedPostalCode = []
RepeatedIteration = []
NewIteration = []
LastIteration = []
for i in range(l):
    if pcl[i] in NewPostalCodes:
            RepeatedPostalCode.append(pcl[i])
            iminusone = i - 1
            iplusone = i + 1
            RepeatedIteration.append(iminusone)
            RepeatedIteration.append(i)
            if pcl[i] == pcl[iminusone]:
                NewIteration.append(iminusone)
            if pcl[i] != pcl[iplusone]:
                LastIteration.append(i)
    if pcl[i] not in NewPostalCodes:
            NewPostalCodes.append(pcl[i])

In [6]:
l = len(pcl) - 1
PostalCodes =[]
Boroughs = []
Neighborhoods = []
for i in range(l):
    if i in RepeatedIteration:
        if i in NewIteration:
            nlist = []
        nlist.append(nl[i])
        if i in LastIteration:
            PostalCodes.append(pcl[i])
            Boroughs.append(bl[i])
            Neighborhoods.append(nlist)
    if i not in RepeatedIteration:
        PostalCodes.append(pcl[i])
        Boroughs.append(bl[i])
        Neighborhoods.append(nl[i])

In [7]:
column_names = ['PostalCode', 'Borough', 'Neighborhood']
nt = pd.DataFrame({'PostalCode': PostalCodes, 'Borough': Boroughs, 'Neighborhood': Neighborhoods}, columns=column_names)
nstr = nt.Neighborhood.astype(str).str.replace('\[|\]|\'', '')
neighborhood_table = pd.DataFrame({'PostalCode': PostalCodes, 'Borough': Boroughs, 'Neighborhood': nstr}, columns=column_names)
neighborhood_table

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queens Park,Queens Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


#### Try using Geocode to add postal code latitudes and longitudes to dataframe

In [28]:
!pip install geocoder

In [29]:
import geocoder 

In [30]:
l = len(PostalCodes) - 1
Latitudes = []
Longitudes = []
for i in range(l):
    postal_code = PostalCodes[i]
    print (postal_code)
    lat_lng_coords = None
    print (lat_lng_coords)
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        print (lat_lng_coords)

M3A
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


KeyboardInterrupt: 

#### Use .csv to add postal code latitudes and longitudes to dataframe since Geocoder is not working

In [11]:
df = pd.read_csv("https://cocl.us/Geospatial_data")
df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [17]:
neighborhoods_lat_long = neighborhood_table.merge(df, left_on='PostalCode', right_on='Postal Code')
toronto_neighborhoods = neighborhoods_lat_long.drop('Postal Code', axis=1)
toronto_neighborhoods

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queens Park,Queens Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
